### Fake news detection problem

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import re
import string
import pickle

In [2]:
df_fake = pd.read_csv("data/Fake.csv")
df_true = pd.read_csv("data/True.csv")

In [3]:
df_fake.head(10)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


In [4]:
df_true.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [5]:
df_fake.shape, df_true.shape

((23481, 4), (21417, 4))

In [6]:
df_fake["class"] = 0
df_true["class"] = 1

In [7]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(23400,23470, -1):
    df_fake.drop([i], axis=0, inplace=True)
df_true_manual_testing = df_true.tail(10)
for i in range(21416, 21400, -1):
    df_true.drop([i], axis=0, inplace=True)

In [8]:
df_manual_testing = pd.concat([df_fake_manual_testing, df_true_manual_testing], axis=0)
df_manual_testing.to_csv("manual_testing.csv")

In [9]:
df_merge = pd.concat([df_fake, df_true], axis = 0)
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [10]:
df = df_merge.drop(["title", "subject", "date"], axis=1)
df.head(10)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


In [11]:
df = df.sample(frac=1)

In [12]:
df.head(10)

,text,class
4012,Donald Trump is a petty thin-skinned wannabe d...,0
9478,"In her new book Hacks , Donna Brazile express...",0
1096,"Ever since his presidential campaign, Donald T...",0
20785,So now it s funny that Bill Clinton had oral s...,0
22784,IMAGE: Globalist scribe for the American branc...,0
17090,Obama couldn t just endorse Joe Biden because ...,0
2804,With Donald Trump writing Executive Orders tha...,0
8788,"Much like all of his other speeches, President...",0
12857,WARSAW (Reuters) - Poland is ready to defend i...,1
14236,HARARE (Reuters) - Zimbabwe s former vice pres...,1


In [13]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [14]:
def word_drop(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '',text)
    text = re.sub("\\W", " ",text)
    text = re.sub('https?://\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [15]:
df["text"] = df["text"].apply(word_drop)

In [16]:
df.head(10)

,text,class
4012,donald trump is a petty thin skinned wannabe d...,0
9478,in her new book hacks donna brazile express...,0
1096,ever since his presidential campaign donald t...,0
20785,so now it s funny that bill clinton had oral s...,0
22784,image globalist scribe for the american branc...,0
17090,obama couldn t just endorse joe biden because ...,0
2804,with donald trump writing executive orders tha...,0
8788,much like all of his other speeches president...,0
12857,warsaw reuters poland is ready to defend i...,1
14236,harare reuters zimbabwe s former vice pres...,1


In [17]:
x = df["text"]
y = df["class"]

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = .25)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

#### Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [23]:
LR.score(xv_test, y_test)

0.9874342750200517

In [24]:
pred_LR = LR.predict(xv_test)

In [25]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.99      0.99      0.99      5326

    accuracy                           0.99     11221
   macro avg       0.99      0.99      0.99     11221
weighted avg       0.99      0.99      0.99     11221



#### Decision Tree classification

In [26]:
from sklearn.tree import DecisionTreeClassifier

In [27]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [28]:
DT.score(xv_test, y_test)

0.995900543623563

In [29]:
pred_DT = DT.predict(xv_test)

In [30]:
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5895
           1       1.00      1.00      1.00      5326

    accuracy                           1.00     11221
   macro avg       1.00      1.00      1.00     11221
weighted avg       1.00      1.00      1.00     11221



#### Gradient Boosting Classifier

In [31]:
from sklearn.ensemble import GradientBoostingClassifier

In [32]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

KeyboardInterrupt: 

In [ ]:
GBC.score(xv_test, y_test)

0.9964352553248373

In [ ]:
pred_GBC = GBC.predict(xv_test)

In [ ]:
print(classification_report(y_test, pred_GBC))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5876
           1       0.99      1.00      1.00      5345

    accuracy                           1.00     11221
   macro avg       1.00      1.00      1.00     11221
weighted avg       1.00      1.00      1.00     11221



#### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
RFC.score(xv_test,y_test)

0.9885928170394795

In [ ]:
pred_RFC = RFC.predict(xv_test)

In [ ]:
print(classification_report(y_test, pred_RFC))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5876
           1       0.99      0.99      0.99      5345

    accuracy                           0.99     11221
   macro avg       0.99      0.99      0.99     11221
weighted avg       0.99      0.99      0.99     11221



#### Manual Testing

In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(word_drop) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [ ]:
pickle.dump(DT, open('model.pkl', 'wb'))

In [ ]:
news = str(input())
manual_testing(news)



LR Prediction: Not Fake News 
DT Prediction: Not Fake News 
GBC Prediction: Not Fake News 
RFC Prediction: Not Fake News
